In [3]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats

In [4]:
%matplotlib widget

In [63]:
# This function defines the test statistic for the counting experiment. 
# There are some 
def q_s(s,b,k):
    
    mask_k_zero = (k==0)
    mask_k_lt_b = (k<=b) & np.invert(mask_k_zero)
    mask_k_gt_b = (k>b) & np.invert(mask_k_zero)
    
    output = np.zeros(len(k))
    if s+b==0:
        return output
    
    k_zero = k[mask_k_zero]
    output[mask_k_zero] = 2 * (s)
    
    k_lt_b = k[mask_k_lt_b]
    output[mask_k_lt_b] = 2 * (s - k_lt_b*(np.log(s+b) - np.log(b)))

    k_gt_b = k[mask_k_gt_b]
    output[mask_k_gt_b] = 2 * (s  - (k_gt_b - b) - \
                                          k_gt_b*( np.log(s+b) - np.log(k_gt_b) ) )

    return output


def q_s2(s,b,k):
    
    s_hat = k-b
    
    if s_hat>s:
        return 0
    else:
        if s_hat<=0:
            return 2 * (s   - k*( np.log(s+b) - np.log(b)))
        else:
            return 2 * (s  - (k - b) - k*( np.log(s+b) - np.log(k)))

In [64]:
_,ax = plt.subplots(1,2,figsize=(9.2,4), sharey=True)

lamda_test = 2
bck = 0
test1 = np.random.poisson(lam=(lamda_test+bck),size=2000)
qtest1 = q_s(lamda_test,bck,test1)
qtest2 = np.array([q_s2(lamda_test,bck,k_) for k_ in test1])

ax[0].hist(qtest1,bins=np.linspace(0.,10.,40),log=True,color='r',label='Original')
ax[0].set(xlabel=r'Value of $q_s$', ylabel='Number of events (2000 total)')
ax[1].hist(qtest2,bins=np.linspace(0.,10.,40),log=True,color='b',label=r'New $q_s$')
ax[1].set(xlabel=r'Value of $q_s$', ylabel='Number of events (2000 total)')

x = np.arange(0, 10, .05)
[a.plot(x, 0.5*stats.chi2.pdf(x, df=1)*2000/40*10, color='r', lw=2, label='scaled chi square') for a in ax]
[a.legend() for a in ax]
sum(qtest2>0.3)/2000

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/analysis_user/.local/lib/python3.6/site-packages/ipykernel_launcher.py:17: RuntimeWarning: divide by zero encountered in log
/home/analysis_user/.local/lib/python3.6/site-packages/ipykernel_launcher.py:34: RuntimeWarning: divide by zero encountered in log
/home/analysis_user/.local/lib/python3.6/site-packages/ipykernel_launcher.py:34: RuntimeWarning: invalid value encountered in multiply


0.2635

In [61]:
test1[:10], qtest2[:10]

(array([4, 4, 3, 1, 1, 1, 3, 2, 1, 0]),
 array([0.        , 0.        , 0.        , 0.61370564, 0.61370564,
        0.61370564, 0.        , 0.        , 0.61370564,        nan]))

In [65]:
_,ax = plt.subplots(1,2,figsize=(9.2,4), sharey=True)
ax[0].scatter(test1,qtest1)
ax[0].set(title='old construction', xlabel='Observed events', ylabel=r'$q_{N=2.3}$')
ax[1].scatter(test1,qtest2)
ax[1].set(title='new construction', xlabel='Observed events')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[Text(0.5, 1.0, 'new construction'), Text(0.5, 0, 'Observed events')]

In [52]:
num_hypotheses = 250

hypotheses = np.linspace(0.,30,num_hypotheses)
limits_prob = np.zeros(num_hypotheses)
limits_90 = np.zeros(num_hypotheses)
limits_90_prob = np.zeros(num_hypotheses)

limits_prob_np = np.zeros(num_hypotheses)
limits_95_np = np.zeros(num_hypotheses)
limits_95_prob = np.zeros(num_hypotheses)

num_sim = 10000
for i in range(num_hypotheses):
    if i % 50 == 0:
        print(i)
    hyp = hypotheses[i]
    
    test = np.random.poisson(lam=(hyp+0),size=num_sim)
    qtest = q_s(hyp,0,test)
    qtest2 = np.array([q_s2(hyp,0,k_) for k_ in test])

    limits_prob[i] = sum(qtest>2.71)/num_sim
    limits_90[i] = np.quantile(qtest,0.90)
    limits_90_prob[i] = sum(qtest>limits_90[i])/num_sim
    
    limits_prob_np[i] = sum(qtest2>1.64)/num_sim
    limits_95_np[i] = np.quantile(qtest2,0.67)

0


/home/analysis_user/.local/lib/python3.6/site-packages/ipykernel_launcher.py:36: RuntimeWarning: divide by zero encountered in log
/home/analysis_user/.local/lib/python3.6/site-packages/ipykernel_launcher.py:36: RuntimeWarning: invalid value encountered in double_scalars
/home/analysis_user/.local/lib/python3.6/site-packages/ipykernel_launcher.py:17: RuntimeWarning: divide by zero encountered in log
/home/analysis_user/.local/lib/python3.6/site-packages/ipykernel_launcher.py:36: RuntimeWarning: invalid value encountered in multiply


50
100
150
200


In [54]:
_,ax = plt.subplots(1,2,figsize=(9.2,4), sharey=True)

ax[0].plot(hypotheses,limits_prob,'b',label='Old construction')
ax[0].plot(hypotheses,limits_prob_np,'r',label='New construction')
ax[0].plot(hypotheses,limits_90_prob,'g',label='Old construction - prob')

ax[0].plot(hypotheses,hypotheses*0+0.1,'g--')
ax[0].legend()
ax[0].set(xlim=(10,30),ylim=(0,0.4),xlabel='Signal hypothesis (num counts)', ylabel=r'$p-value$')

ax[1].plot(hypotheses,limits_prob,'b',label='Old construction')
ax[1].plot(hypotheses,limits_prob_np,'r',label='New construction')
ax[1].plot(hypotheses,limits_90_prob,'g',label='Old construction - prob')
ax[1].plot(hypotheses,hypotheses*0+0.1,'g--')
ax[1].legend()
ax[1].set(xlim=(0,10),ylim=(0,0.4),xlabel='Signal hypothesis (num counts)')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[(0.0, 10.0), (0.0, 0.4), Text(0.5, 0, 'Signal hypothesis (num counts)')]

In [53]:
_,ax = plt.subplots()
ax.plot(hypotheses,limits_90,'g',label='90% old')
plt.plot(hypotheses,limits_95_np,'b',label='90% new')
plt.plot(hypotheses,limits_95_np*0+2.71,'r--',label='2.71 ')

ax.set(xlim=(0.,10.), ylim=(0.,12.), xlabel='Signal hypothesis (num counts)',ylabel=r'Critical value $q_s^*$')
ax.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …